# Climate-related classification tasks on ChatGPT - using techniques for better prompt engineering
<hr>
<h3>In this Notebook we have used various techniques for better prompt engineering in order to get better results, including chaining of prompts, using different patterns for prompts like <i>"The Persona Pattern"</i> where which enables
the model to take a certain point of view or role, in our
case, a climate, sustainability, and environmental expert; <i>“The Fact Check List Pattern”</i>, which instructs the model
to output the most important points of a text and then use
those points as the input in a follow-up prompt and the
<i>“Reflection Pattern”</i> in which the model is asked to
explain the reasoning behind its response.</h3>
<h3>For that purpose, in this script we have created a function that connects to the OpenAI API using the API key and sends the data and the prompts provided to ChatGPT in batch, in chosen batch size. Additionally, a function that chains prompts was also created such that it receives the first prompt and forwards it as input in the second prompt.</h3>
<h3>In order to be able to run the scripts and the tasks, first the OpenAI key needs to be set.</h3>
<h3>Because this key is a secret and gives access to your OpenAI account, it should be hidden and not available in plain text to the public. It is advised to store these keys in files on your computer on on some cloud, like Google Drive where other people cannot access them and then open them in the Notebook and set the keys via variables, that way they can be protected from the public.</h3>
<h3>In our approach, we used text files on Google Drive to store the key and we open them in the Notebook, set the appropriate variable and then use the variable to set the key.</h3>
<hr>
<h3>To use this script, you need to set your OpenAI key, to do that, if you use the same approach as us, first you need to store your key in a file and store them on Google Drive and after that only the path to the file in which the key is stored needs to be changed and the script will work.</h3>
<h3>Alternative approaches include uploading your locally stored files to the Colab Notebook, using a GitHub repository or using alternative storage solutions.</h3>
<h3>On the following link you can find ways to deal with your files on various storage providers: <a href="https://neptune.ai/blog/google-colab-dealing-with-files">https://neptune.ai/blog/google-colab-dealing-with-files</a></h3>
<hr>
<h3>Each task is structured in its own Colab Notebook and in order to get the results for a task, first the appropriate keys must be set in the Notebook and after that the whole Notebook can just be run and the results will be displayed at the end of the section, either by collapsing the section and running the cells from the whole section at once or running each cell one by one. Some steps are optional, for example saving the results in a .csv file and may be skipped.</h3>

In [ ]:
#This code is for mounting your Google Drive to the Notebook. The path where you can access your whole Google Drive is /content/drive
#Alternatively, the Google Drive may be mounted by clicking the folder icon on the left side menu and then clicking the third icon from the
#left, the dark icon with a folder and the logo of Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-llm

#If you use the same approach as us, with Google Drive, you need to change the paths to your relevant files where the keys are stored on Google Drive

with open('Here put the path to your OpenAI API key', 'r') as file1:
    key = file1.readline()

with open('Here put the path to your OpenAI organization key', 'r') as file2:
    org_key = file2.readline()



from skllm.config import SKLLMConfig

#Alternatively, you can just insert your keys as plain text in the appropriate places, but this is not advised since your keys would be visible to anyone who has access to your Notebook
#For using other approaches, please visit the link provided in the description above that instructs use and import of files from other storage solutions

SKLLMConfig.set_openai_key(key)
SKLLMConfig.set_openai_org(org_key)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00


In [ ]:
import json
import time
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    size = batch_size
    text_list = np.array_split(target_texts, 3)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        print("prompt", p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        print(result, r)
        rez.append(r)
        try:
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez
def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys


def batch_gpt_chained(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + text}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys



def batch_gpt_resp_as_part_prompt(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    summarized_points = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        summarized_points.append(r1)
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + r1}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys, summarized_points




def batch_gpt_verify_cot_prompts(start_prompt,end_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = start_prompt + text + end_prompt
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            parts = r.split("\"")
            i = len(parts)-2
            b = parts[i]
            b = b.lower()
            if b.startswith("yes"):
              b = 1
            else:
              b = 0
        except:
            b = -1
            print("error parsing"+r)
        # print(i, len(p), "-", r, b, end=": ")
        print(f"{i}: \| response: {r} \| result: {b}\n\n")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys

<h1>Running the Classification of paragraphs into one of the recommended categories by the Task Force on Climate-Related Financial Disclosures (TCFD) on ChatGPT</h1>
<h4>In this task, paragraphs are classified into one category from the TCFD Recommended categories for climate change-related texts.</h4>
<hr>
<h4>Classification classes:</h4>
<h4>0 - is not climate-related</h4>
<h4>1 - is about metrics</h4>
<h4>2 - is about strategy</h4>
<h4>3 - is about risk</h4>
<h4>4 - is about governance</h4>
<hr>
<h4>First, the required library - datasets is loaded in order to be able to work with the dataset and the corresponding dataset is downloaded from HuggingFace and loaded into the dataset variable.</h4>


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/tcfd_recommendations")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--tcfd_recommendations-8f7123f770abbd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

<h4>After that, the paragraphs and the labels are extracted from the dataset and are loaded into a Pandas DataFrame that allows easier manipulation with the data and better visualization of the data, with tables.</h4>

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 2], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 3], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,2
1,"Verizon’s environmental, health and safety man...",3
2,"In 2019, the Company closed a series of transa...",2
3,"In December 2020, the AUC approved the Electri...",2
4,"Finally, there is a reputational risk linked t...",2
...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",2
396,Climate change is producing changes in weather...,2
397,A sound and certain regulatory and fiscal envi...,0
398,"Across our global workforce, 20% of Gold Field...",0


<h4>The initial prompt that was sent to ChatGPT was the following: "You are the sustainability, environment, and climate change expert. Read the following paragraph and extract the most important points from the text and return only the points and their explanations:"</h4>
<h4>After the response was received, the response was then used as the input for the second prompt in which ChatGPT is instructed to perform the classification. The chained prompt was the following: "Read the following points and answer only with the overall class of which all points are summarized without any explanations. Answer only with 0 if the text is not climate-related, answer only with 1 if the text is about metrics for sustainability, environment, and climate change, answer only with 2 if the text is about strategy for sustainability, environment, and climate change, answer only with 3 if the text is about risk for sustainability, environment, and climate change and answer only with 4 if the text is about governance for sustainability, environment, and climate change:"</h4>
<hr>
<h4>The results were received both in a numerical - categorical and textual representation</h4>

In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following paragraph and extract the most important points from the text and return only the points and their explanations: \n\n'
chain_prompt = 'Read the following points and answer only with the overall class of which all points are summarized without any explanations. Answer only with 0 if the text is not climate-related, answer only with 1 if the text is about metrics for sustainability, environment, and climate change, answer only with 2 if the text is about strategy for sustainability, environment, and climate change, answer only with 3 if the text is about risk for sustainability, environment, and climate change and answer only with 4 if the text is about governance for sustainability, environment, and climate change: \n\n'
texts = df["text"].to_list()
summarized_points = []
rez, rez_keys, summarized_points = batch_gpt_resp_as_part_prompt(prompt,chain_prompt, texts, 1)

1 431 - 2 2: 2 889 - 3 3: 3 818 - 0 0: 4 1131 - 4 4: 5 621 - 3 3: 6 590 - 1 1: 7 587 - 3 3: 8 635 - 1. 0
2. 0
3. 1 1: 9 731 - 4 4: 10 506 - 1 1: 
11 795 - 4 4: 12 462 - 3 3: 13 474 - 1 1: 14 708 - 3 (risk for sustainability, environment, and climate change) 3: 15 428 - 1 1: 16 637 - 0 0: 17 487 - 1 1: 18 648 - 1 1: 19 571 - 3 3: 20 402 - 3 3: 
21 878 - 3 3: 22 637 - 0 0: 23 913 - 1 (metrics for sustainability, environment, and climate change) 1: 24 858 - 1 1: 25 813 - 3 3: error calling API, retrying...
error parsing- Point: Increased severity/frequency of extreme weather events
- Explanation: Refers to the worsening and more frequent occurrence of natural disasters such as hurricanes, floods, and wildfires.

- Point: Loss or impairment of key manufacturing sites
- Explanation: Refers to the damage or destruction of facilities where products are manufactured, leading to disruptions in supply chains and production.

- Point: Inability to procure sufficient raw materials
- Explanation: R

<h4>The predictions that are received, are stored in a variable, then they are stored both in numerical and textual representations in a Pandas DataFrame to further be compared and evaluated.</h4>

In [ ]:
rez

['2',
 '3',
 '0',
 '4',
 '3',
 '1',
 '3',
 '1. 0\n2. 0\n3. 1',
 '4',
 '1',
 '4',
 '3',
 '1',
 '3 (risk for sustainability, environment, and climate change)',
 '1',
 '0',
 '1',
 '1',
 '3',
 '3',
 '3',
 '0',
 '1 (metrics for sustainability, environment, and climate change)',
 '1',
 '3',
 '- Point: Increased severity/frequency of extreme weather events\n- Explanation: Refers to the worsening and more frequent occurrence of natural disasters such as hurricanes, floods, and wildfires.\n\n- Point: Loss or impairment of key manufacturing sites\n- Explanation: Refers to the damage or destruction of facilities where products are manufactured, leading to disruptions in supply chains and production.\n\n- Point: Inability to procure sufficient raw materials\n- Explanation: Refers to the difficulty in obtaining necessary resources due to disruptions in transportation or other barriers.\n\n- Point: Disruption to transportation of raw materials or finished goods\n- Explanation: Refers to difficulties

In [ ]:
rez_keys

[2,
 3,
 0,
 4,
 3,
 1,
 3,
 1,
 4,
 1,
 4,
 3,
 1,
 3,
 1,
 0,
 1,
 1,
 3,
 3,
 3,
 0,
 1,
 1,
 3,
 -1,
 2,
 1,
 4,
 3,
 0,
 2,
 2,
 3,
 2,
 1,
 -1,
 2,
 3,
 1,
 3,
 1,
 2,
 2,
 2,
 1,
 4,
 3,
 4,
 1,
 4,
 1,
 2,
 1,
 1,
 2,
 3,
 4,
 4,
 1,
 1,
 2,
 3,
 3,
 1,
 2,
 4,
 3,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 3,
 0,
 2,
 3,
 0,
 1,
 1,
 0,
 2,
 4,
 2,
 2,
 2,
 3,
 2,
 0,
 3,
 1,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 4,
 1,
 0,
 1,
 1,
 4,
 3,
 4,
 1,
 3,
 3,
 0,
 3,
 1,
 4,
 1,
 3,
 3,
 0,
 0,
 4,
 1,
 4,
 1,
 2,
 3,
 3,
 3,
 1,
 3,
 4,
 2,
 1,
 3,
 2,
 2,
 1,
 3,
 4,
 2,
 1,
 3,
 3,
 3,
 2,
 1,
 3,
 4,
 4,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 2,
 1,
 2,
 4,
 1,
 4,
 1,
 1,
 3,
 2,
 2,
 3,
 3,
 1,
 4,
 2,
 2,
 1,
 1,
 3,
 1,
 3,
 3,
 3,
 4,
 0,
 4,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 4,
 1,
 1,
 4,
 1,
 4,
 2,
 4,
 3,
 4,
 1,
 1,
 3,
 3,
 1,
 1,
 4,
 1,
 4,
 3,
 3,
 1,
 4,
 0,
 3,
 3,
 1,
 4,
 4,
 1,
 2,
 1,
 1,
 0,
 2,
 3,
 1,
 4,
 1,
 3,
 3,
 1,
 4,
 3,
 4,
 4,
 1,
 4,
 2,
 3,
 2,
 4,
 2,
 3,
 4,
 4

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,2,2,2
1,"Verizon’s environmental, health and safety man...",3,3,3
2,"In 2019, the Company closed a series of transa...",2,0,0
3,"In December 2020, the AUC approved the Electri...",2,4,4
4,"Finally, there is a reputational risk linked t...",2,3,3
...,...,...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",2,1,1
396,Climate change is producing changes in weather...,2,1,1
397,A sound and certain regulatory and fiscal envi...,0,-1,to investment \n0 (not climate-related)
398,"Across our global workforce, 20% of Gold Field...",0,1,1


<h4>The labels that couldn't be automatically mapped were mapped manually by observing their textual counterparts and seeing where they belong.</h4>

In [ ]:
df['gpt-label'].value_counts()

 1    126
 3    110
 2     65
 4     58
 0     36
-1      5
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations
25,2 Increased severity/frequency of extreme weat...,2,-1,- Point: Increased severity/frequency of extre...
36,"BB DTVM, by strategic direction, performs exte...",3,-1,"Class 2: Metrics for sustainability, environme..."
259,"At Group level, in 2019 there were 61 spills o...",3,-1,"- In 2019, there were 61 spills of hazardous m..."
374,Report a description of how their investment p...,2,-1,Overall class: 1
397,A sound and certain regulatory and fiscal envi...,0,-1,to investment \n0 (not climate-related)


In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,3)

In [ ]:
df['gpt-label'].value_counts()

1    126
3    115
2     65
4     58
0     36
Name: gpt-label, dtype: int64

<h4>The received summarized points were then stored in the DataFrame, such that they can be used as input in the Scikit LLM library</h4>

In [ ]:
summarized_points

["Points:\n1. Sustainable strategy range\n2. Proprietary 'red lines'\n3. Ensuring poorest-performing companies not eligible for investment\n\nExplanation:\n1. The text mentions a sustainable strategy range, which suggests that the company in question has a focus on sustainability and is implementing strategies to achieve this goal.\n2. The company uses proprietary 'red lines,' or criteria, to determine which companies are eligible for investment. These criteria likely focus on environmental, social, and governance (ESG) performance, which are important markers of sustainability.\n3. By excluding the poorest-performing companies from investment, the company is prioritizing sustainability and sending a message that it takes ESG factors seriously. This can help drive positive change within the broader investment community and promote better practices among companies.",
 "Points:\n1. Verizon has an environmental, health, and safety management system \n2. The system identifies, controls, an

In [ ]:
df['points'] = summarized_points

<h4>In the following step, the Zero Shot classifier is imported, the paragraphs are stored into variable X and the correct labels in variable Y. The labels are provided to the classifier and the classification process begins with the paragraphs being sent to the model.</h4>

In [ ]:
from skllm import ZeroShotGPTClassifier

X = df['points']
Y = df['label']

candidate_labels = [
    "the provided text is not about sustainability, environment and climate change",
    "the provided text is about metrics for sustainability, environment and climate change",
    "the provided text is about strategy for sustainability, environment and climate change",
    "the provided text is about risk for sustainability, environment and climate change",
    "the provided text is about governance for sustainability, environment and climate change"
]


clf = ZeroShotGPTClassifier()
clf.fit(None, [candidate_labels])
preds = clf.predict(X)

100%|██████████| 400/400 [10:42<00:00,  1.61s/it]


<h4>The predictions that are received, are stored in a variable, then they are stored both in numerical and textual representations in a Pandas DataFrame to further be compared and evaluated.</h4>

In [ ]:
preds

['the provided text is about strategy for sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is about metrics for sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is about governance for sustainability, environment and climate change',
 'the provided text is about risk for sustainability, environment and climate change',
 'the provided text is not about sustainability, environment and climate change',
 'the provided text is about risk for sustainabili

In [ ]:
df['gpt-explanations'] = preds

In [ ]:
labels = []

for label in preds:
  if label == "the provided text is not about sustainability, environment and climate change":
    labels.append(0)
  elif label == "the provided text is about metrics for sustainability, environment and climate change":
    labels.append(1)
  elif label == "the provided text is about strategy for sustainability, environment and climate change":
    labels.append(2)
  elif label == "the provided text is about risk for sustainability, environment and climate change":
    labels.append(3)
  elif label == "the provided text is about governance for sustainability, environment and climate change":
    labels.append(4)


labels

[2,
 3,
 0,
 0,
 3,
 1,
 3,
 0,
 4,
 3,
 0,
 3,
 1,
 3,
 0,
 0,
 0,
 1,
 3,
 3,
 3,
 0,
 3,
 1,
 3,
 3,
 1,
 1,
 4,
 3,
 1,
 3,
 2,
 3,
 1,
 1,
 1,
 4,
 3,
 2,
 3,
 1,
 2,
 1,
 1,
 2,
 4,
 3,
 1,
 0,
 4,
 1,
 4,
 3,
 1,
 2,
 3,
 1,
 0,
 1,
 1,
 2,
 3,
 3,
 0,
 2,
 0,
 3,
 1,
 3,
 3,
 0,
 3,
 0,
 3,
 3,
 1,
 2,
 3,
 0,
 3,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 3,
 1,
 0,
 3,
 1,
 3,
 3,
 3,
 1,
 1,
 0,
 1,
 4,
 2,
 0,
 0,
 2,
 2,
 3,
 2,
 1,
 3,
 3,
 0,
 3,
 0,
 1,
 3,
 3,
 3,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 3,
 3,
 0,
 3,
 3,
 4,
 1,
 1,
 3,
 1,
 2,
 1,
 3,
 4,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 4,
 3,
 1,
 3,
 1,
 1,
 3,
 3,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 1,
 3,
 3,
 3,
 4,
 0,
 4,
 1,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 3,
 1,
 2,
 1,
 1,
 4,
 1,
 0,
 2,
 0,
 3,
 3,
 0,
 3,
 0,
 3,
 1,
 0,
 4,
 1,
 1,
 3,
 0,
 2,
 1,
 0,
 3,
 3,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 3,
 1,
 0,
 0,
 3,
 3,
 2,
 0,
 3,
 4,
 3,
 1,
 4,
 2,
 3,
 2,
 0,
 2,
 3,
 4,
 2,


In [ ]:
df['gpt-label'] = labels

In [ ]:
df

,text,label,gpt-label,gpt-explanations,points
0,Sustainable strategy ‘red lines’ For our susta...,2,2,the provided text is about strategy for sustai...,Points:\n1. Sustainable strategy range\n2. Pro...
1,"Verizon’s environmental, health and safety man...",3,3,the provided text is about risk for sustainabi...,"Points:\n1. Verizon has an environmental, heal..."
2,"In 2019, the Company closed a series of transa...",2,0,"the provided text is not about sustainability,...",Points:\n- The Company sold its Canadian fossi...
3,"In December 2020, the AUC approved the Electri...",2,0,"the provided text is not about sustainability,...",Points:\n- AUC approved deferral of distributi...
4,"Finally, there is a reputational risk linked t...",2,3,the provided text is about risk for sustainabi...,- Reputational risk for oil companies \n- Poss...
...,...,...,...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",2,2,the provided text is about strategy for sustai...,Points:\n- Banco do Brasil Foundation celebrat...
396,Climate change is producing changes in weather...,2,3,the provided text is about risk for sustainabi...,1. Climate change leads to changes in environm...
397,A sound and certain regulatory and fiscal envi...,0,0,"the provided text is not about sustainability,...",Points:\n1. Regulatory and fiscal environment ...
398,"Across our global workforce, 20% of Gold Field...",0,1,the provided text is about metrics for sustain...,- Gold Fields' global workforce has 20% women ...


In [ ]:
df['gpt-label'].value_counts()

3    123
1    113
0     87
2     53
4     24
Name: gpt-label, dtype: int64

<h4>The DataFrame is also stored on Google Drive, for later viewing and analysis. This step can be skipped.</h4>

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/scikit-llm/chatgpt_climate_tcfd_recommendations_scikit_llm.csv",index=False)

<h4>In the following section, the predicted labels are compared to the actual labels and the results are displayed.</h4>
<hr>
<h4>In the first row of the output, three metrics are displayed in the following order: <h6>(precision, recall, fscore, support - optional, may be none)</h6></h4>
<h4>In the second row, only the F1 Score is displayed, for better clarity.</h4>
<h4>In the third row the confusion matrix is displayed.</h4>
<h4>In the fourth row the whole classification report is displayed, with the metrics per class: precision, recall, f1 score and support; the accuracy, per class and overall and the macro and micro averages of each metric.</h4>

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.45759588424961695, 0.5132752804362666, 0.4198191064265232, None)
0.4198191064265232
[[51  2  2 21  4]
 [ 5 38  5  1  0]
 [30 56 40 63  8]
 [ 0 14  2 29  3]
 [ 1  3  4  9  9]]
              precision    recall  f1-score   support

           0       0.59      0.64      0.61        80
           1       0.34      0.78      0.47        49
           2       0.75      0.20      0.32       197
           3       0.24      0.60      0.34        48
           4       0.38      0.35      0.36        26

    accuracy                           0.42       400
   macro avg       0.46      0.51      0.42       400
weighted avg       0.58      0.42      0.40       400

